## Train

In [45]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import random
import pickle
import cv2
import os
%matplotlib inline

In [46]:
args = {"dataset":"dataset", "model":"spider_vgg.model", "labelbin":"lb.pickle"}

In [50]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 10
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

In [24]:
# initialize the data and labels
data = []
labels = []

In [26]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)

In [27]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [28]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("data matrix: {:.2f}MB".format(data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 80% of the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

data matrix: 246.67MB


In [39]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, 
                         shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

In [41]:
# initialize the model
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [51]:
# train the network
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

Epoch 1/10
28/28 [==============================] - 13s 474ms/step - loss: 0.1202 - acc: 0.9576 - val_loss: 1.3691 - val_acc: 0.7598
Epoch 2/10
28/28 [==============================] - 14s 487ms/step - loss: 0.1828 - acc: 0.9365 - val_loss: 1.5319 - val_acc: 0.7118
Epoch 3/10
28/28 [==============================] - 13s 479ms/step - loss: 0.1473 - acc: 0.9477 - val_loss: 1.5171 - val_acc: 0.7118
Epoch 4/10
28/28 [==============================] - 14s 486ms/step - loss: 0.1567 - acc: 0.9510 - val_loss: 1.6972 - val_acc: 0.6769
Epoch 5/10
28/28 [==============================] - 16s 585ms/step - loss: 0.1617 - acc: 0.9454 - val_loss: 2.0128 - val_acc: 0.6463
Epoch 6/10
28/28 [==============================] - 15s 550ms/step - loss: 0.1408 - acc: 0.9510 - val_loss: 1.3929 - val_acc: 0.7336
Epoch 7/10
28/28 [==============================] - 16s 569ms/step - loss: 0.1334 - acc: 0.9542 - val_loss: 1.2783 - val_acc: 0.7817
Epoch 8/10
28/28 [==============================] - 16s 574ms/step - 

In [52]:
# save the model to disk
model.save(args["model"])

In [ ]:
# save the label binarizer to disk
f = open(args["labelbin"], "wb")
f.write(pickle.dumps(lb))
f.close()

## Classify

In [87]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

args = {"image":"test/Funnel_Web (214).jpg", "model":"spider_vgg.model", "labelbin":"lb.pickle"}

# load the image
image = cv2.imread(args["image"])
output = image.copy()
 
# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [ ]:
# load the trained convolutional neural network and the label binarizer
model = load_model(args["model"])
lb = pickle.loads(open(args["labelbin"], "rb").read())

In [88]:
# classify the input image
proba = model.predict(image)[0]
idx = np.argmax(proba)
label = lb.classes_[idx]

In [89]:
label

'Sydney_Funnel_Web_Spider'